In [ ]:
!pip install tbats


In [ ]:
!pip install pmdarima


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


## Importing Libraries

In [ ]:
# Standard library imports
import os
import pickle

# Third-party library imports
import numpy as np  # Numerical operations
import pandas as pd  # Data manipulation and analysis
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Statistical data visualization

# Scikit-learn imports for preprocessing and model evaluation
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler  # Data scaling
from sklearn.model_selection import train_test_split  # Splitting data into train and test sets
from sklearn.metrics import mean_squared_error  # Model evaluation metric

# Statsmodels imports for time series analysis
from statsmodels.tsa.stattools import adfuller, kpss  # Statistical tests for stationarity
from statsmodels.tsa.arima.model import ARIMA  # ARIMA model
from statsmodels.tsa.statespace.sarimax import SARIMAX  # SARIMAX model
import statsmodels.api as sm  # General statsmodels API

# Pmdarima imports for automated ARIMA modeling
from pmdarima import auto_arima
import pmdarima as pm

# TBATS model for time series forecasting
from tbats import TBATS
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [ ]:
import warnings
warnings.filterwarnings("ignore")


## Load your dataset

In [ ]:
data = pd.read_csv('../../DataSet/expanded_dataset.csv')


### Look at Dataset

In [ ]:
data.sample(5)


#### Converting  'Datetime' column to datetime

In [ ]:
# data['Datetime'] = pd.to_datetime(data['Datetime'], format='%m/%d/%Y %H:%M', dayfirst=True)
data['Datetime'] = pd.to_datetime(data['Datetime'],  errors='coerce')
# format='%Y-%m-%d %H:%M:%S',


Set the 'Dates' column as the index

In [ ]:
data.set_index('Datetime', inplace=True)


In [ ]:
data['TotalPowerConsumption']= data['PowerConsumption_Zone1'] + data['PowerConsumption_Zone2'] + data['PowerConsumption_Zone3']


In [ ]:
data= data.drop(['PowerConsumption_Zone1','PowerConsumption_Zone2','PowerConsumption_Zone3'],axis=1)


## Checking for Missing values

In [ ]:
data.isnull().sum()


In [ ]:
data.tail()


In [ ]:
print('Index frequency:', pd.infer_freq(data.index)) #https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
print('Number of missing values:', data.isnull().sum())


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_numeric_boxplots(df, cols_per_row=3):
    """
    Create box plots for numeric columns in a DataFrame, arranged in multiple rows.

    Parameters:
        df (DataFrame): The input DataFrame.
        cols_per_row (int): Number of plots per row.

    Returns:
        None
    """
    # Select numeric columns
    numeric_cols = df.select_dtypes(include='number')
    num_cols = len(numeric_cols.columns)
    rows = (num_cols // cols_per_row) + (num_cols % cols_per_row > 0)

    # Create subplots
    fig = make_subplots(
        rows=rows, cols=cols_per_row,
        subplot_titles=numeric_cols.columns
    )

    # Add box plot traces
    for i, col in enumerate(numeric_cols.columns):
        row = (i // cols_per_row) + 1
        col_position = (i % cols_per_row) + 1
        fig.add_trace(
            go.Box(y=numeric_cols[col], name=col, boxmean=True),
            row=row, col=col_position
        )

    # Update layout
    fig.update_layout(
        height=rows * 400,  # Adjust height dynamically based on the number of rows
        width=cols_per_row * 200,
        title_text="Box Plots for Numeric Columns",
        showlegend=False,
        template="plotly_dark"
    )

    # Show the figure
    fig.show()


In [ ]:
plot_numeric_boxplots(data, cols_per_row=6)


## Checking for Outliers

Some Outliers found. Lets remove them

In [ ]:
RS = RobustScaler()


In [ ]:
x = data.drop(['TotalPowerConsumption'],axis=1)
y = data['TotalPowerConsumption']


In [ ]:
plot_numeric_boxplots(x, cols_per_row=5)



In [ ]:
scaled_x = RS.fit_transform(x)


In [ ]:
x= pd.DataFrame(scaled_x, columns=x.columns)


In [ ]:
plot_numeric_boxplots(x, cols_per_row=5)


In [ ]:
sns.boxplot(x='TotalPowerConsumption',data=data)
plt.title('Outlier Detection in TotalPwerConsumption')
plt.show()


In [ ]:
Q1 = data['TotalPowerConsumption'].quantile(0.25)
Q3 = data['TotalPowerConsumption'].quantile(0.75)

IQR = Q3 - Q1

lowerbound = Q1 - (1.5 * IQR)
upperbound = Q3 + (1.5 * IQR)

data = data[(data.TotalPowerConsumption >= lowerbound) & (data.TotalPowerConsumption <= upperbound)]


In [ ]:
sns.boxplot(x='TotalPowerConsumption',data=data)
plt.title('Outlier Detection in TotalPwerConsumption')
plt.show()


## Checking and Fixing Data Distribution

In [ ]:


def plot_distribution(data, column):
    """
    Plot the distribution of a specified column using Plotly.

    Parameters:
    data (DataFrame): The data containing the column.
    column (str): The column name to plot.
    """
    # Create distribution plot
    # fig = ff.create_distplot(
    #     [data[column]], [column], show_hist=True, show_rug=True,
    #     colors=['#636EFA']  # Add color to the plot
    # )

    # Update layout
    # fig.update_layout(
    #     title=f'Distribution Plot of {column}',
    #     xaxis_title=column,
    #     yaxis_title='Density',
    #     width=1000,
    #     height=600,
    #     plot_bgcolor='white',  # Set background color to white
    #     paper_bgcolor='white',  # Set paper background color to white
    #     font=dict(color='black')  # Set font color to black
    # )

    # # Update x and y axis
    # fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
    # fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

    # fig.show()

# Example usage
# plot_distribution(data, 'TotalPowerConsumption')


In [ ]:
sns.displot(data['Temperature'], kde=True)
plt.title('Distribution Plot of Temperature')
plt.show()


In [ ]:
plot_distribution(data, 'Temperature')


In [ ]:
data['Temperature'].skew()


In [ ]:
from scipy.stats import boxcox
data['Temperature'], lambda_boxcox = boxcox(data['Temperature'])


In [ ]:
data['Temperature'].skew()


In [ ]:
plot_distribution(data, 'Temperature')


In [ ]:
# sns.displot(data['Humidity'], kde=True)
# plt.title('Distribution Plot of Humidity')
# plt.show()
plot_distribution(data, 'Humidity')


In [ ]:
data['Humidity'].skew()


In [ ]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='yeo-johnson')
data['Humidity'] = pt.fit_transform(data[['Humidity']])


In [ ]:
data['Humidity'].skew()


In [ ]:
# sns.displot(data['Humidity'], kde=True)
# plt.title('Distribution Plot of Humidity')
# plt.show()
plot_distribution(data, 'Humidity')


In [ ]:
# sns.displot(data['WindSpeed'], kde=True)
# plt.title('Distribution Plot of Windspeed')
# plt.show()
plot_distribution(data, 'WindSpeed')


In [ ]:
data['WindSpeed'].skew()


In [ ]:
from scipy.stats import boxcox
data['WindSpeed'], lambda_boxcox = boxcox(data['WindSpeed'])


In [ ]:
data['WindSpeed'].skew()


In [ ]:
# sns.displot(data['WindSpeed'], kde=True)
# plt.title('Distribution Plot of Windspeed')
# plt.show()
plot_distribution(data, 'WindSpeed')


In [ ]:
# sns.displot(data['GeneralDiffuseFlows'], kde=True)
# plt.title('Distribution Plot of General Diffuse Flow')
# plt.show()
plot_distribution(data, 'GeneralDiffuseFlows')


In [ ]:
data['GeneralDiffuseFlows'].skew()


In [ ]:
data['GeneralDiffuseFlows'], lambda_boxcox = boxcox(data['GeneralDiffuseFlows'])


In [ ]:
data['GeneralDiffuseFlows'].skew()


In [ ]:
# sns.displot(data['GeneralDiffuseFlows'], kde=True)
# plt.title('Distribution Plot of General Diffuse Flow')
# plt.show()
plot_distribution(data, 'GeneralDiffuseFlows')


In [ ]:
# sns.displot(data['DiffuseFlows'], kde=True)
# plt.title('Distribution Plot of Diffuse Flow')
# plt.show()
plot_distribution(data, 'DiffuseFlows')


In [ ]:
data['DiffuseFlows'].skew()


In [ ]:
data['DiffuseFlows'], lambda_boxcox = boxcox(data['DiffuseFlows'])


In [ ]:
data['DiffuseFlows'].skew()


In [ ]:
# sns.displot(data['DiffuseFlows'], kde=True)
# plt.title('Distribution Plot of Diffuse Flow')
# plt.show()
plot_distribution(data, 'DiffuseFlows')


In [ ]:
sns.displot(data['TotalPowerConsumption'], kde=True)
plt.title('Distribution Plot of TotalPowerConsumption')
plt.show()


In [ ]:
data['TotalPowerConsumption'].skew()


In [ ]:
data['TotalPowerConsumption'], lambda_boxcox = boxcox(data['TotalPowerConsumption'])


In [ ]:
data['TotalPowerConsumption'].skew()


In [ ]:
plot_numeric_boxplots(data, cols_per_row=3)


In [ ]:
x= data.drop(['TotalPowerConsumption'],axis=1)
y=data['TotalPowerConsumption']


In [ ]:
import plotly.graph_objects as go

# Filter the data for the specified date range
def plot_consumption_over_period():
    filtered_data = y['2018-12-01 00:00:00':]

    # Create the figure
    fig = go.Figure()

    # Add the trace
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data, mode='lines', line=dict(color='red')))

    # Update the layout
    fig.update_layout(
        title='Total power consumption time series',
        xaxis_title='Datetime',
        yaxis_title='Power Consumption',
        width=1800,
        height=700
    )

    # Show the figure
    fig.show()


In [ ]:
plot_consumption_over_period()


In [ ]:
def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(dftest[0:4], index=["Test Statistic", "p-value",
                                             "Lags Used", "Number of Observations Used"])
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

def kpss_test(timeseries):
    print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"])
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    print(kpss_output)


## Scaling

In [ ]:
Scaler = StandardScaler()


In [ ]:
x_scaled = Scaler.fit_transform(x)


In [ ]:
x_scaled


In [ ]:
x_scaled.shape


In [ ]:
y.shape


In [ ]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,shuffle=False)


#### AdFuller Report

In [ ]:
adf_test(y_train)
print('\nDifferentiations needed according to ADF:', pm.arima.utils.ndiffs(y_train, test='adf'))
print('\n')
kpss_test(y_train)
print('\nDifferentiations needed according to KPSS:', pm.arima.utils.ndiffs(y_train, test='kpss'))


In [ ]:
fas_d = sm.tsa.acf(y, nlags=200)
fas_s = sm.tsa.acf(y.resample('D').mean(), nlags=30)

fig, axs = plt.subplots(1, 2, figsize=(15,7))
fig.suptitle('Time series correlogram', y=1)
axs[0].stem(fas_d)
axs[0].set_title('ACF')
axs[0].set_xlabel('n_lags')
axs[0].grid(True)
axs[1].stem(fas_s)
axs[1].set_title('ACF (diary average)')
axs[1].set_xlabel('n_lags / n_days')
axs[1].grid(True)
plt.tight_layout()
plt.show()


## ARIMA

In [ ]:
if os.path.exists('../Models/search_arima_model_auto.pkl'):
    print("Model already trained and saved at", '../Models/search_arima_model_auto.pkl')
    with open('../Models/search_arima_model_auto.pkl', 'rb') as pkl_file:
        a_auto_model = pickle.load(pkl_file)
else:
    a_auto_model = pm.auto_arima(y_train, start_p=1, start_q=1, d=1,D=1,
                                    max_p=10, max_q=10, max_d=10,
                                    max_order=None,
                                    m=7,
                                    seasonal=False,
                                    test='adf',
                                    n_jobs=-1,
                                    trace=True,
                                    error_action='ignore')

    with open('../Models/search_arima_model_auto.pkl', 'wb') as pkl_file:
        pickle.dump(a_auto_model, pkl_file)



In [ ]:
a_auto_model.fit(y_train)


In [ ]:
arima_pred = a_auto_model.predict(n_periods=len(y_test))


In [ ]:
print(a_auto_model.summary())


In [ ]:
mse = mean_squared_error(y_test, arima_pred)
print(f'Mean Squared Error: {mse}')


In [ ]:
def calculate_all_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    print(f'Mean Squared Error: {mse}')
    print(f'Mean Absolute Error: {mae}')
    print(f'Root Mean Squared Error: {rmse}')
    print(f'R-squared: {r2}')
    
    return mse, mae, rmse, r2


In [ ]:
calculate_all_metrics(y_test, arima_pred)


## SRIMAX

In [ ]:
def SARIMAX(tru):
  if os.path.exists('../Models/search_sarima_model_auto.pkl') and tru:
      print("Model already trained and saved at", '../Models/search_sarima_model_auto.pkl')
      with open('../Models/search_sarima_model_auto.pkl', 'rb') as pkl_file:
          auto_model = pickle.load(pkl_file)
  else:
    auto_model = pm.auto_arima(
        y_train,
        seasonal=True,
        d=1,
        m=24,
        trace=True,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True,
        stationary=False,
        start_p=1,
        start_q=1,
        start_P=0,
        seasonal_test='ocsb'
    )
    with open('../Models/search_sarima_model_auto.pkl', 'wb') as pkl_file:
      pickle.dump(auto_model, pkl_file)
  return auto_model


In [ ]:
auto_model=SARIMAX(True)


In [ ]:
auto_model.fit(y_train)


In [ ]:
print(auto_model.summary())


In [ ]:

def plot_residuals(auto_model):
    """
    Plot the residuals of the Auto ARIMA model using Plotly.

    Parameters:
    auto_model (object): The fitted Auto ARIMA model.
    """
    # Calculate residuals
    residuals = auto_model.resid()

    # Create figure
    fig = go.Figure()

    # Plot residuals
    fig.add_trace(go.Scatter(
        x=residuals.index,
        y=residuals,
        mode='lines',
        name='Residuals',
        line=dict(color='blue')
    ))

    # Update layout
    fig.update_layout(
        title='Residuals of the Auto ARIMA Model',
        xaxis_title='Index',
        yaxis_title='Residuals',
        height=600,
        width=1000
    )

    fig.show()

# Example usage
# plot_residuals(auto_model)


In [ ]:
# plot_residuals(auto_model)


In [ ]:
# with open('search_sarima_model_auto.pkl', 'rb') as pkl_file:
#     auto_model = pickle.load(pkl_file)


In [ ]:
sarimax_pred = auto_model.predict(n_periods=len(y_test))


In [ ]:
calculate_all_metrics(y_test, sarimax_pred)


In [ ]:
mse = mean_squared_error(y_test, sarimax_pred)
print(f'Mean Squared Error (SARIMAX): {mse}')


In [ ]:
distinct_counts = len(np.unique(sarimax_pred))
print(distinct_counts)


In [ ]:
import plotly.graph_objects as go

def evaluate_regression_models(x_test, y_test, arima_pred, sarimax_pred, step=5000):
    """
    Evaluate regression models and plot the results using Plotly.

    Parameters:
    x_test (pd.Index): Index of the test set.
    y_test (pd.Series): Actual values of the test set.
    arima_pred (pd.Series): Predicted values from the ARIMA model.
    sarimax_pred (pd.Series): Predicted values from the SARIMAX model.
    step (int): Step size for downsampling the data for plotting.
    """
    # Downsample data
    x_subset = x_test[::step]
    y_subset = y_test[::step]
    arima_subset = arima_pred[::step]
    sarimax_subset = sarimax_pred[::step]

    # Create figure
    fig = go.Figure()

    # Plot actual values
    fig.add_trace(go.Scatter(
        x=x_subset,
        y=y_subset,
        mode='lines+markers',
        name='Actual',
        line=dict(color='black'),
        marker=dict(symbol='circle', size=6, opacity=0.6)
    ))

    # Plot ARIMA forecast
    fig.add_trace(go.Scatter(
        x=x_subset,
        y=arima_subset,
        mode='lines+markers',
        name='ARIMA Forecast',
        line=dict(color='blue', dash='dash'),
        marker=dict(symbol='square', size=6, opacity=0.8)
    ))

    # Plot SARIMAX forecast
    fig.add_trace(go.Scatter(
        x=x_subset,
        y=sarimax_subset,
        mode='lines+markers',
        name='SARIMAX Forecast',
        line=dict(color='orange', dash='dot'),
        marker=dict(symbol='diamond', size=6, opacity=0.8)
    ))

    # Update layout
    fig.update_layout(
        title='Forecast vs Actuals',
        xaxis_title='Time Periods',
        yaxis_title='Power Consumption',
        xaxis=dict(tickangle=30),
        legend=dict(x=0, y=1),
        height=600,
        width=1000,
        xaxis_showgrid=True,
        yaxis_showgrid=True,
        xaxis_gridcolor='rgba(0,0,0,0.1)',
        yaxis_gridcolor='rgba(0,0,0,0.1)',
        xaxis_griddash='dash',
        yaxis_griddash='dash'
    )

    fig.show()

# Example usage
# evaluate_regression_models(x_test, y_test, arima_pred, sarimax_pred)


In [ ]:
evaluate_regression_models(y_test.index, y_test, arima_pred, sarimax_pred, step=5000)


In [ ]:
# # Plotting results
# step = 5000

# plt.figure(figsize=(14, 7))
# plt.plot(x_test.index[::step], y_test[::step], label='Actual', color='black')  # Actual values
# plt.plot(x_test.index[::step], arima_pred[::step], label='ARIMA Forecast', color='blue')  # ARIMA forecast
# plt.plot(x_test.index[::step], sarimax_pred[::step], label='SARIMAX Forecast', color='orange')  # SARIMAX forecast
# plt.title('Forecast vs Actuals')
# plt.xlabel('Date')
# plt.ylabel('Power Consumption')
# plt.legend()
# plt.show()


In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# # Downsample every 10th point (adjust as needed)
# step = 5000
# x_subset = x_test.index[::step]
# y_subset = y_test[::step]
# arima_subset = arima_pred[::step]
# sarimax_subset = sarimax_pred[::step]

# # Set figure size
# plt.figure(figsize=(12, 6))

# # Plot actual values
# plt.plot(x_subset, y_subset, label='Actual', color='black', linestyle='-', marker='o', markersize=3, alpha=0.6)

# # Plot ARIMA forecast
# plt.plot(x_subset, arima_subset, label='ARIMA Forecast', color='blue', linestyle='--', marker='s', markersize=3, alpha=0.8)

# # Plot SARIMAX forecast
# plt.plot(x_subset, sarimax_subset, label='SARIMAX Forecast', color='orange', linestyle='-.', marker='d', markersize=3, alpha=0.8)

# # Adding titles and labels
# plt.title('Forecast vs Actuals')
# plt.xlabel('Time Periods')
# plt.ylabel('Power Consumption')

# # Improve x-tick readability
# plt.xticks(rotation=30, ha='right')

# plt.legend()
# plt.grid(True, linestyle='--', alpha=0.5)  # Add grid for better readability
# plt.tight_layout()
# plt.show()


In [ ]:

# # Downsample every 10th point (adjust as needed)
# step = 5000
# x_subset = x_test.index[::step]
# y_subset = y_test[::step]
# arima_subset = arima_pred[::step]
# sarimax_subset = sarimax_pred[::step]

# # Create figure
# fig = go.Figure()

# # Plot actual values
# fig.add_trace(go.Scatter(
#     x=x_subset,
#     y=y_subset,
#     mode='lines+markers',
#     name='Actual',
#     line=dict(color='black'),
#     marker=dict(symbol='circle', size=6, opacity=0.6)
# ))

# # Plot ARIMA forecast
# fig.add_trace(go.Scatter(
#     x=x_subset,
#     y=arima_subset,
#     mode='lines+markers',
#     name='ARIMA Forecast',
#     line=dict(color='blue', dash='dash'),
#     marker=dict(symbol='square', size=6, opacity=0.8)
# ))

# # Plot SARIMAX forecast
# fig.add_trace(go.Scatter(
#     x=x_subset,
#     y=sarimax_subset,
#     mode='lines+markers',
#     name='SARIMAX Forecast',
#     line=dict(color='orange', dash='dot'),
#     marker=dict(symbol='diamond', size=6, opacity=0.8)
# ))

# # Update layout
# fig.update_layout(
#     title='Forecast vs Actuals',
#     xaxis_title='Time Periods',
#     yaxis_title='Power Consumption',
#     xaxis=dict(tickangle=30),
#     legend=dict(x=0, y=1),
#     height=600,
#     width=1000,
#     xaxis_showgrid=True,
#     yaxis_showgrid=True,
#     xaxis_gridcolor='rgba(0,0,0,0.1)',
#     yaxis_gridcolor='rgba(0,0,0,0.1)',
#     xaxis_griddash='dash',
#     yaxis_griddash='dash'
# )

# fig.show()


In [ ]:
# # Prepare the x-axis positions
# x_positions = np.arange(len(x_test))

# # Create the bar width
# bar_width = 0.25

# # Plotting results using bar charts
# plt.figure(figsize=(14, 7))

# # Actual values
# plt.bar(x_positions, y_test, width=bar_width, label='Actual', color='black', alpha=0.6)

# # ARIMA forecast
# plt.bar(x_positions + bar_width, arima_pred, width=bar_width, label='ARIMA Forecast', color='blue', alpha=0.6)

# # SARIMAX forecast
# plt.bar(x_positions + 2 * bar_width, sarimax_pred, width=bar_width, label='SARIMAX Forecast', color='orange', alpha=0.6)

# # Adding titles and labels
# plt.title('Forecast vs Actuals')
# plt.xlabel('Time Periods')
# plt.ylabel('Power Consumption')
# plt.xticks(x_positions + bar_width, x_test.index.date, rotation=45)  # Rotate date labels for better readability
# plt.legend()
# plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels
# plt.show()


In [ ]:
# # Select the last 'num_obs' observations
# num_obs = 10  # Adjust as needed
# x_test_subset = x_test.iloc[-num_obs:]  # Keep the original index
# y_test_subset = y_test.iloc[-num_obs:]
# arima_pred_subset = arima_pred[-num_obs:]
# sarimax_pred_subset = sarimax_pred[-num_obs:]

# # Adjust x_positions
# bar_width = 0.2
# spacing = 0.1
# x_positions = np.arange(num_obs) * (bar_width * 3 + spacing)

# plt.figure(figsize=(14, 7))

# # Actual values
# plt.bar(x_positions, y_test_subset, width=bar_width, label='Actual', color='black', alpha=0.6)

# # ARIMA forecast
# plt.bar(x_positions + bar_width + spacing, arima_pred_subset, width=bar_width, label='ARIMA Forecast', color='blue', alpha=0.6)

# # SARIMAX forecast
# plt.bar(x_positions + 2 * (bar_width + spacing), sarimax_pred_subset, width=bar_width, label='SARIMAX Forecast', color='orange', alpha=0.6)

# # Labels and display adjustments
# plt.title('Forecast vs Actuals')
# plt.xlabel('Time Periods')
# plt.ylabel('Power Consumption')
# plt.xticks(x_positions + (bar_width + spacing), x_test_subset.index.date, rotation=45)  # Use index directly
# plt.legend()
# plt.tight_layout()
# plt.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Select the last 'num_obs' observations
num_obs = 10  # Adjust as needed
x_test_subset = x_test.iloc[-num_obs:]  # Keep the original index
y_test_subset = y_test.iloc[-num_obs:]
arima_pred_subset = arima_pred[-num_obs:]
sarimax_pred_subset = sarimax_pred[-num_obs:]

# Adjust x_positions
bar_width = 0.2
spacing = 0.1
x_positions = np.arange(num_obs)

# Create figure
fig = go.Figure()

# Actual values
fig.add_trace(go.Bar(
    x=x_positions,
    y=y_test_subset,
    name="Actual",
    marker_color="black",
    opacity=0.6
))

# ARIMA forecast
fig.add_trace(go.Bar(
    x=x_positions + (bar_width + spacing),
    y=arima_pred_subset,
    name="ARIMA Forecast",
    marker_color="blue",
    opacity=0.6
))

# SARIMAX forecast
fig.add_trace(go.Bar(
    x=x_positions + 2 * (bar_width + spacing),
    y=sarimax_pred_subset,
    name="SARIMAX Forecast",
    marker_color="orange",
    opacity=0.6
))

# Layout adjustments
fig.update_layout(
    title="Forecast vs Actuals",
    xaxis=dict(
        tickmode="array",
        tickvals=x_positions + (bar_width + spacing),
        ticktext=x_test_subset.index.date,
    ),
    yaxis_title="Power Consumption",
    barmode="group",
    bargap=spacing,
    template="plotly_white"
)

# Show plot
fig.show()


In [ ]:
train = y[:'2018-09-30 23:50:00']
test = y['2018-10-01 00:00:00':]


In [ ]:
y.tail()


In [ ]:
#MODEL
estimator = TBATS(seasonal_periods=[144, 1008], use_trend=False)

if os.path.exists('../Models/search_tvats_model_auto.pkl'):
    print("Model already trained and saved at", '../Models/search_tvats_model_auto.pkl')
    with open('../Models/search_tvats_model_auto.pkl', 'rb') as pkl_file:
        fitted_model = pickle.load(pkl_file)
else:
  fitted_model = estimator.fit(train)
  with open('../Models/search_tvats_model_auto.pkl', 'wb') as pkl_file:
    pickle.dump(fitted_model, pkl_file)


In [ ]:

print('TBATS model summary:\n', fitted_model.summary())

pred_train = pd.Series(data=fitted_model.y_hat, index=train.index)

pred_test, conf_test = fitted_model.forecast(steps=len(test), confidence_level=0.95)
pred_test = pd.Series(data=pred_test, index=test.index)
conf_test = pd.DataFrame(data={'lower_bound':conf_test['lower_bound'], 'upper_bound':conf_test['upper_bound']}, index=test.index)

err_train = pd.Series(data=fitted_model.resid, index=train.index)
err_test = test - pred_test
err = pd.concat([err_train, err_test])


In [ ]:
print(len(test))  # Ensure it's a positive integer


In [ ]:
calculate_all_metrics(test, pred_test)


In [ ]:


# # Whole visualization
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=train.index,
#     y=train,
#     mode='lines',
#     name='Train',
#     line=dict(color='red')
# ))

# fig.add_trace(go.Scatter(
#     x=test.index,
#     y=test,
#     mode='lines',
#     name='Test',
#     line=dict(color='indianred')
# ))

# fig.add_trace(go.Scatter(
#     x=pred_train.index,
#     y=pred_train,
#     mode='lines',
#     name='Train pred',
#     line=dict(color='teal')
# ))

# fig.add_trace(go.Scatter(
#     x=pred_test.index,
#     y=pred_test,
#     mode='lines',
#     name='Test pred',
#     line=dict(color='turquoise')
# ))

# fig.add_trace(go.Scatter(
#     x=conf_test.index,
#     y=conf_test['lower_bound'],
#     mode='lines',
#     name='Lower Bound',
#     line=dict(width=0),
#     showlegend=False
# ))

# fig.add_trace(go.Scatter(
#     x=conf_test.index,
#     y=conf_test['upper_bound'],
#     mode='lines',
#     name='Upper Bound',
#     line=dict(width=0),
#     fill='tonexty',
#     fillcolor='rgba(0,0,0,0.1)',
#     showlegend=True
# ))

# fig.update_layout(
#     title='TBATS forecasting',
#     xaxis_title='Datetime',
#     yaxis_title='Power Consumption',
#     legend=dict(x=0, y=1),
#     height=600,
#     width=1000
# )

# fig.show()

# # TBATS errors
# fig = make_subplots(rows=1, cols=2, subplot_titles=('TBATS errors', 'Error Distribution'))

# # Error plot
# fig.add_trace(go.Scatter(
#     x=err.index,
#     y=err,
#     mode='markers',
#     name='Errors',
#     marker=dict(color='blue')
# ), row=1, col=1)

# fig.update_xaxes(title_text='Date', row=1, col=1)
# fig.update_yaxes(title_text='Power Consumption', row=1, col=1)

# # Error histogram
# fig.add_trace(go.Histogram(
#     x=err,
#     nbinsx=13,
#     name='Error Distribution',
#     marker=dict(color='blue')
# ), row=1, col=2)

# fig.update_xaxes(title_text='Power Consumption', row=1, col=2)

# fig.update_layout(
#     height=600,
#     width=1200,
#     showlegend=False,
#     title_text='TBATS errors'
# )

# fig.show()


In [ ]:
# # Train dataset and forecasting visualization
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=train['2018-8-01 00:00:00':'2018-9-15 23:50:00'].index,
#     y=train['2018-8-01 00:00:00':'2018-9-15 23:50:00'],
#     mode='lines',
#     name='Train',
#     line=dict(color='red')
# ))

# fig.add_trace(go.Scatter(
#     x=pred_train['2018-8-01 00:00:00':'2018-9-15 23:50:00'].index,
#     y=pred_train['2018-8-01 00:00:00':'2018-9-15 23:50:00'],
#     mode='lines',
#     name='Train pred',
#     line=dict(color='teal')
# ))

# fig.update_layout(
#     title='TBATS forecasting (train)',
#     xaxis_title='Datetime',
#     yaxis_title='Power consumption',
#     legend=dict(x=0, y=1),
#     height=600,
#     width=1000
# )

# fig.show()

# # TBATS errors (train)
# fig = make_subplots(rows=1, cols=2, subplot_titles=('TBATS errors (train)', 'Error Distribution'))

# # Error plot
# fig.add_trace(go.Scatter(
#     x=err['2017-03-01 00:00:00':'2017-03-06 23:50:00'].index,
#     y=err['2017-03-01 00:00:00':'2017-03-06 23:50:00'],
#     mode='markers',
#     name='Errors',
#     marker=dict(color='blue')
# ), row=1, col=1)

# fig.update_xaxes(title_text='Datetime', row=1, col=1)
# fig.update_yaxes(title_text='Power Consumption', row=1, col=1)

# # Error histogram
# fig.add_trace(go.Histogram(
#     x=err['2017-03-01 00:00:00':'2017-03-06 23:50:00'],
#     nbinsx=13,
#     name='Error Distribution',
#     marker=dict(color='blue')
# ), row=1, col=2)

# fig.update_xaxes(title_text='Power Consumption', row=1, col=2)

# fig.update_layout(
#     height=600,
#     width=1200,
#     showlegend=False,
#     title_text='TBATS errors (train)'
# )

# fig.show()
